In [51]:
import numpy as np

In [52]:
import pandas as pd


df = pd.read_csv(r'././data/aida_competition.csv')


df

In [53]:
# df[['Diver_'], ['Nationality']] = df['Diver'].apply(lambda x: pd.Series(x.rsplit('(', 1))) 
df[['Diver', 'Nationality']] = df["Diver"].apply(lambda x: pd.Series(str(x).split("(", 1)))

df['Nationality'] = df['Nationality'].apply(lambda x: x[:-1]) # remove the last parenthesis

In [54]:
df

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,...,Title Event,Event Type,Day,Category Event,Month,Season,Experience dive,Experience discipline,Remarks_norm,Nationality
0,1,Tasos Grillakis,M,FIM,NaN,00:00,33,23 m,YELLOW,12.0,...,Depth Event 2016,Depth Competition,2016-07-17,other,7,Summer,1,1,NaN,GRC
1,2,Antonis Papantonatos,M,FIM,NaN,00:00,55,47 m,YELLOW,38.0,...,Depth Event 2016,Depth Competition,2016-07-17,other,7,Summer,1,1,NaN,GRC
2,3,Dimitris Koumoulos,M,CNF,NaN,00:00,55,55 m,WHITE,55.0,...,Depth Event 2016,Depth Competition,2016-07-17,other,7,Summer,1,1,NaN,GRC
3,4,Christos Papadopoulos,M,CWT,NaN,00:00,55,55 m,WHITE,55.0,...,Depth Event 2016,Depth Competition,2016-07-17,other,7,Summer,1,1,NaN,GRC
4,5,Anna Chalari,F,CWT,NaN,00:00,15,15 m,WHITE,15.0,...,Depth Event 2016,Depth Competition,2016-07-17,other,7,Summer,1,1,NaN,GRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26837,7,Karine Le Flanchec,F,CWTB,1.0,09:40,57,57 m,WHITE,57,...,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other,7,Summer,4,4,NaN,FRA
26838,8,Anne-Sophie Passalboni,F,CWT,1.0,09:50,52,52 m,WHITE,52,...,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other,7,Summer,11,1,NaN,FRA
26839,9,Clementine Marie,F,CNF,1.0,10:00,40,40 m,WHITE,40,...,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other,7,Summer,21,7,NaN,FRA
26840,10,Eisve Treciakauskaite,F,CWT,1.0,10:10,33,28 m,YELLOW,22,...,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other,7,Summer,4,4,NO TAG,LTU


In [55]:

df['Remarks_norm'] = df['Remarks'].apply(lambda x: str(x).lower())
cond_list = [df['Remarks_norm'].str.contains('|'.join(['bo', 'blackout']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['early']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['pull']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['record', 'national', 'nr']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['grab']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['airway']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['no tag']), case=False),
             df['Remarks_norm'].str.contains('|'.join(['dqsp']), case=False)
             ]
choice_list = ['DQBO', 'EARLY TURN', 'DQPULL', 'NR', 'GRAB', 'DQAIRWAYS', 'NO TAG', 'DQSP']
df['Remarks_norm'] = np.select(cond_list, choice_list, default='')

print(len(df['Remarks'].unique()))
remarks_df = sorted([x for x in df['Remarks_norm'].unique()])
print(len(remarks_df))
df[['Remarks','Remarks_norm']].to_csv('remarks.csv')

409
9


In [56]:
df['Category Event'] = df['Title Event'].apply(lambda x: x.lower())
cond_list = [df['Category Event'].str.contains('|'.join(['vertical blue']), case=False),
             df['Category Event'].str.contains('|'.join(['nac', 'nice abyss contest']), case=False),
             df['Category Event'].str.contains('|'.join(['world']), case=False),
             df['Category Event'].str.contains('|'.join(['panglao']), case=False)
             ]
choice_list = ['VB', 'NAC', 'World Championship', 'Panglao']
df['Category Event'] = np.select(cond_list, choice_list, default='other')

print(len(df['Title Event'].unique()))
remarks_df = sorted([x for x in df['Title Event'].unique()])
print(len(remarks_df))
df[['Title Event','Category Event']].to_csv('events.csv')
pd.DataFrame(remarks_df).to_csv('cat_events.csv')
# pd.DataFrame(sorted([x for x in df['Category Event'].unique()])).to_csv('cat_events.csv')

766
766


In [57]:
df['Month'] = pd.to_datetime(df['Day']).dt.month 

labels=['Winter', 'Spring', 'Summer', 'Fall']
doy = pd.to_datetime(df['Day']).dt.dayofyear
df['Season'] = pd.cut(doy + 11 - 366*(doy > 355), bins=[0, 91, 183, 275, 366], labels=labels)

df['Experience dive'] = df.groupby('Diver').cumcount() + 1
df['Experience discipline'] = df.groupby(['Diver', 'Discipline']).cumcount() + 1
# df['Experience event'] = df.groupby(['Diver', 'Title Event']).cumcount() + 1
df[df['Diver'] == 'Clementine Marie (FRA)']
# df['Diver'] == 'Abdel Alouach'
# df
# df['Remarks_norm'] = df['Remarks'].apply(lambda x: x.lower())
# df['AP'] = df['AP'].apply(lambda x: x.split()[0])
df.to_csv('aida_competition_clean.csv', index=False)

In [58]:
df.head()

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,...,Title Event,Event Type,Day,Category Event,Month,Season,Experience dive,Experience discipline,Remarks_norm,Nationality
0,1,Tasos Grillakis,M,FIM,NaN,00:00,33,23 m,YELLOW,12.0,...,Depth Event 2016,Depth Competition,2016-07-17,other,7,Summer,1,1,,GRC
1,2,Antonis Papantonatos,M,FIM,NaN,00:00,55,47 m,YELLOW,38.0,...,Depth Event 2016,Depth Competition,2016-07-17,other,7,Summer,1,1,,GRC
2,3,Dimitris Koumoulos,M,CNF,NaN,00:00,55,55 m,WHITE,55.0,...,Depth Event 2016,Depth Competition,2016-07-17,other,7,Summer,1,1,,GRC
3,4,Christos Papadopoulos,M,CWT,NaN,00:00,55,55 m,WHITE,55.0,...,Depth Event 2016,Depth Competition,2016-07-17,other,7,Summer,1,1,,GRC
4,5,Anna Chalari,F,CWT,NaN,00:00,15,15 m,WHITE,15.0,...,Depth Event 2016,Depth Competition,2016-07-17,other,7,Summer,1,1,,GRC


In [59]:
cond = 'Remark_norm' == 'DBQO'
list_col = ['Experience dive'] #'Discipline', 'AP', 'Gender', 

for col in list_col :
    count_discipline = df[col].value_counts() 
    count_discipline_cond = df[df['Remarks_norm'] == 'DQBO'][col].value_counts() 
    total = count_discipline*100 /df.shape[0]
    prop = count_discipline_cond * 100/ count_discipline
    moy_exclu_max = prop[prop != prop.iloc[prop.argmax()]].mean()
    print(count_discipline*100 /df.shape[0])
    print(prop.iloc[prop.argmax()])
    print(moy_exclu_max)
    print(prop/moy_exclu_max)


Experience dive
1      17.327323
2      12.856717
3       9.049251
4       6.776693
5       5.394531
         ...    
124     0.007451
136     0.003726
137     0.003726
138     0.003726
139     0.003726
Name: count, Length: 139, dtype: float64
100.0
9.01785221909648
Experience dive
1       0.228887
2       0.179945
3       0.278483
4       0.310910
5       0.306329
         ...    
135          NaN
136          NaN
137          NaN
138          NaN
139    11.089115
Name: count, Length: 139, dtype: float64


In [60]:
# df.groupby(['Diver','Discipline']).agg(['count'])

In [61]:
gk = df.groupby('Diver')

In [62]:
first_compet = gk['Day'].first()
last_compet = gk['Day'].last()
nb_dives = gk['Title Event'].count()
nb_compets = gk['Title Event'].nunique()


In [63]:
# df.groupby(['Diver', 'Remarks_norm']).count()['Remarks_norm']
# gk.apply(lambda x: x[x['Remarks_norm'] == 'DQBO'].count())
nb_bo=df.groupby('Diver')['Remarks_norm'].apply(lambda x: (x=='DQBO').sum()).reset_index(name='count')
nb_red=df.groupby('Diver')['Card'].apply(lambda x: (x=='RED').sum()).reset_index(name='count')


In [64]:
ap_compet = gk['AP'].min(), gk['AP'].max(), gk['AP'].mean()

In [65]:
nb_bo.values[:, 1]

array([4, 0, 0, ..., 0, 0, 0], dtype=object)

In [66]:
gender = gk['Gender'].first()
gender

Diver
                      F
Aaron Kunert          M
Aaron Wood            M
Abdalaziz Alsabe      M
Abdallah Moustafa     M
                     ..
Émilie Walsh          F
Ícaro Do Valle        M
圭一 福里                 M
보걸 강                  M
이서 양                  F
Name: Gender, Length: 4651, dtype: object

In [67]:
df['Discipline'].unique()

array(['FIM', 'CNF', 'CWT', 'CWTB'], dtype=object)

In [68]:
def bo_disci(row, disci):
    print(row.Discipline)
    if (row.Remarks_norm == 'DQBO') :
        if row.Discipline == disci:
            return 1
        else:
            return 0
    else:
        return 0

In [69]:
df['bidouille'] = df['Discipline'] + df['Remarks_norm']

In [70]:
def my_agg(x):
    names = {
        'Disc=Y': x[x['Discipline']=='CWT'].count()}

    return pd.Series(names)

df.groupby('Diver').apply(my_agg)

,Disc=Y
Diver,
,Start 17 Diver ...
Aaron Kunert,Start 2 Diver ...
Aaron Wood,Start 2 Diver ...
Abdalaziz Alsabe,Start 3 Diver ...
Abdallah Moustafa,Start 1 Diver ...
...,...
Émilie Walsh,Start 1 Diver ...
Ícaro Do Valle,Start 3 Diver ...
圭一 福里,Start 0 Diver ...


In [71]:
dqbo_disci = pd.crosstab(df.Diver, [df.Remarks_norm, df.Discipline])['DQBO']


In [72]:
df___ = (df.assign(A=(df['Remarks_norm']== 'DQBO'))
        .groupby(['Diver', 'Discipline'])['A']
        .agg([('qtd_ok','sum'),('qtd','size')])
        .astype(int)
        .reset_index())

In [73]:

# nb_bo_cwt=df.groupby('Diver').apply(lambda x: (np.where([x['Remarks_norm']=='DQBO' and x['Discipline'] =='CWT'] , True, False)).sum()).reset_index(name='count')
# nb_bo_cwt



# nb_bo_cwt=df.groupby('Diver').apply(lambda x: bo_disci(x, "CWT").sum()).reset_index(name='count')
# nb_bo_cwt

In [88]:
df_diver = pd.DataFrame({ 'diver' : sorted(df['Diver'].unique()),
                          'nationality' : gk['Nnationality'].last(),
                          'gender' : gk['Gender'].last(),
                          'first_compet':first_compet.values,
                          'last_compet':last_compet.values,
                          'nb_compet': nb_compets.values,
                          'nb_bo' : nb_bo.values[:, 1],
                          'nb_bo_cwt' : dqbo_disci['CWT'],
                          'nb_bo_cwtb' : dqbo_disci['CWTB'],
                          'nb_bo_cnf' : dqbo_disci['CNF'],
                          'nb_bo_fim' : dqbo_disci['FIM'],
                          'nb_red' : nb_red.values[:, 1],
                          'nb_dives' : gk['Experience dive'].last(),
                          '%_bo' :  (nb_bo.values[:, 1] *100) /gk['Experience dive'].last(),
                          '%_bo_cwt' : (dqbo_disci['CWT']*100) /gk['Experience dive'].last(),
                          '%_bo_cwtb' : (dqbo_disci['CWTB'] *100)/gk['Experience dive'].last(),
                          '%_bo_cnf' : (dqbo_disci['CNF']*100) /gk['Experience dive'].last(),
                          '%_bo_fim' : (dqbo_disci['FIM']*100)/gk['Experience dive'].last(),
                        
                          })#, last_compet, *ap_compet])
# df_reversed = df_diver[::-1]

In [89]:
df_diver[df_diver['diver']=='Clementine Marie']
df_diver_date = df_diver[df_diver['last_compet'] > "2021-01-01"]
df_diver_date = df_diver_date[df_diver_date['nb_compet'] > 5]
df_diver_date.shape

(223, 16)

In [90]:
df_diver_date.head(20)

,diver,first_compet,last_compet,nb_compet,nb_bo,nb_bo_cwt,nb_bo_cwtb,nb_bo_cnf,nb_bo_fim,nb_red,nb_dives,%_bo,%_bo_cwt,%_bo_cwtb,%_bo_cnf,%_bo_fim
Diver,,,,,,,,,,,,,,,,
Abdelatif Alouach,Abdelatif Alouach,2019-09-09,2023-07-20,10,1,0,1,0,0,2,25,4.0,0.000000,4.000000,0.000000,0.000000
Abdullah Al Omran,Abdullah Al Omran,2017-08-08,2022-08-09,6,1,0,0,0,1,4,16,6.25,0.000000,0.000000,0.000000,6.250000
Aki Shimoyama,Aki Shimoyama,2016-10-10,2023-01-14,11,0,0,0,0,0,1,15,0.0,0.000000,0.000000,0.000000,0.000000
Alejandra Lopez,Alejandra Lopez,2015-04-27,2022-09-25,14,3,1,0,1,1,6,40,7.5,2.500000,0.000000,2.500000,2.500000
Alejandro Lemus,Alejandro Lemus,2016-05-26,2023-04-30,32,9,3,1,2,3,15,116,7.758621,2.586207,0.862069,1.724138,2.586207
Alena Konecna,Alena Konecna,2015-08-28,2022-09-17,10,1,0,1,0,0,2,28,3.571429,0.000000,3.571429,0.000000,0.000000
Alenka Artnik,Alenka Artnik,2015-08-27,2023-06-16,13,2,1,1,0,0,2,30,6.666667,3.333333,3.333333,0.000000,0.000000
Alessia Zecchini,Alessia Zecchini,2014-09-19,2023-03-31,12,3,2,0,0,1,8,38,7.894737,5.263158,0.000000,0.000000,2.631579
Alexandros Koukos,Alexandros Koukos,2016-10-08,2022-09-30,10,1,0,0,0,1,2,18,5.555556,0.000000,0.000000,0.000000,5.555556


In [91]:
df_diver_date.to_csv('diver_stat.csv', index=False)

In [ ]:
df_diver

,diver,first_compet,last_compet,nb_compet,nb_bo,nb_red,nb_dives,percent_bo
Diver,,,,,,,,
(),(),2002-10-31,2001-11-10,18,4,4,26,15.384615
Aaron Kunert (DEU),Aaron Kunert (DEU),2012-08-18,2014-06-28,2,0,0,3,0.0
Aaron Wood (CAN),Aaron Wood (CAN),2004-06-18,2004-10-29,2,0,0,2,0.0
Abdalaziz Alsabe (SAU),Abdalaziz Alsabe (SAU),2016-11-03,2016-08-27,2,2,2,4,50.0
Abdallah Moustafa (LBN),Abdallah Moustafa (LBN),2019-11-09,2019-11-09,1,0,0,1,0.0
...,...,...,...,...,...,...,...,...
Émilie Walsh (CAN),Émilie Walsh (CAN),2017-09-16,2019-09-01,2,0,0,2,0.0
Ícaro Do Valle (BRA),Ícaro Do Valle (BRA),2016-06-04,2019-10-20,4,0,2,18,0.0
圭一 福里 (JPN),圭一 福里 (JPN),2022-09-29,2022-09-30,1,0,1,2,0.0
